In [1]:
from IPython.core.debugger import set_trace

#%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="dark_background")

In [2]:
SENTIMENT_LABELLED_DATA_FILEPATH = '..\Data\Sentiment Analysis Dataset.csv'
DATA_PREPROCESSING_FILEPATH = '../Data Preprocessed'

TOKENIZER_OUTPUT_FILEPATH = 'tokenizer_100K.pickle'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
# MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove_91_X'

Import Dataset

In [3]:
txt_data = pd.read_csv(SENTIMENT_LABELLED_DATA_FILEPATH , sep='|', names=['col1'])

#split data into columns with ','
txt_data = txt_data.col1.str.split(',',  3, expand=True)
txt_data.columns = list(txt_data.iloc[0])
txt_data = txt_data.drop(0)
txt_data.index = np.subtract(txt_data.index, 1)
# x = txt_data.groupby('Sentiment')
# l=[x.get_group(i)['SentimentText'] for i in x.groups]


In [4]:
dataset = pd.concat([txt_data['SentimentText'], txt_data['Sentiment']], axis = 1)
dataset.columns = ['text', 'target']

In [5]:

print(dataset.info(verbose=True))
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577838 entries, 0 to 1577837
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1577838 non-null  object
 1   target  1577838 non-null  object
dtypes: object(2)
memory usage: 36.1+ MB
None


,text,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0


In [6]:
#suffle
dataset = dataset.sample(frac = 1)
dataset

,text,target
331567,@leannenufc Go for it! Chinese sounds yum!,1
1477822,Just came back from Parent/Teacher interviews....,1
1294981,"""Would like to go fishing tonight, but it's so...",0
1337447,Where's the party at tonight? Soooooo bored,0
337831,@lindsaymwalker Better things await you Have...,1
...,...,...
367349,"""@pauliniunia kay,i poseted it.i cant post any...",0
720225,finallly homee....loooooong day.....accompishe...,0
675801,DEADDDDD. wo9rk 11-3,0
552176,:9going to theatretrain now. dont wanna. wannt...,1


In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(DATA_PREPROCESSING_FILEPATH)
from data_preprocess import *

import pickle

In [8]:
text = NLP_preprocess(dataset)
text.preprocess_data()

text.set_tokenizer()
import pickle

# saving
# with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
#     pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
train_dataset = dataset[:100000]
#train_dataset = dataset[:1000000]
train_labels = train_dataset.target.astype(float)
train_padded = text.tokenize_and_pad(train_dataset, train = True)
print(train_padded)
# saving
# with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
#     pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

[[ 13320      7   1301 ...      0      0      0]
 [   317     16  27907 ...      0      0      0]
 [    52      6      7 ...      0      0      0]
 ...
 [106798     45 106799 ...      0      0      0]
 [   194    339  16017 ...      0      0      0]
 [106801    534    222 ...      0      0      0]]


In [10]:
text.set_embedding_matrix()

In [11]:
pd.DataFrame(text.embedding_matrix).info


<bound method DataFrame.info of               0         1         2         3        4        5        6   \
0       0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
1       0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
2      -0.673910  0.544500 -0.448680  0.167630 -0.22777  0.56469  0.23045   
3       0.091552  0.093336 -0.028113  0.369900  0.18956  0.43191  0.10266   
4      -0.111640  0.353960 -0.216360 -0.089658  0.42865 -0.58209  0.23430   
...          ...       ...       ...       ...      ...      ...      ...   
106798  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
106799  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
106800  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
106801  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
106802  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   

             7        8         9   ...    

In [12]:
test_dataset = dataset[:20000]
test_labels = test_dataset.target.astype(float)
test_padded = text.tokenize_and_pad(test_dataset, train = False)
print(test_padded)

[[13320     7  1301 ...     0     0     0]
 [  317    16 27907 ...     0     0     0]
 [   52     6     7 ...     0     0     0]
 ...
 [ 4153    25 43840 ...     0     0     0]
 [  602   179   137 ...     0     0     0]
 [ 7516  4053   946 ...     0     0     0]]


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

# trainable = False
#bidirectional

#Glove
model.add(
    Embedding(
        text.num_words,
        100,
        embeddings_initializer = Constant(text.embedding_matrix),
        input_length = text.max_length,
        trainable = False,
    )
)

#basic
# for i in range(0,200):
# #model.add(Embedding(text.num_words, 8, input_length=text.max_length))
#     model.add(Dense(1000, activation='relu'))
model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           10680300  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 10,760,801
Trainable params: 80,501
Non-trainable params: 10,680,300
_________________________________________________________________


In [15]:
history = model.fit(
    train_padded, train_labels, epochs=40, validation_data=(test_padded, test_labels),
)

Epoch 1/40
3125/3125 [==============================] - 29s 5ms/step - loss: 0.5377 - accuracy: 0.7295 - val_loss: 0.5005 - val_accuracy: 0.7555
Epoch 2/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.5051 - accuracy: 0.7538 - val_loss: 0.4848 - val_accuracy: 0.7653
Epoch 3/40
3125/3125 [==============================] - 14s 5ms/step - loss: 0.4935 - accuracy: 0.7614 - val_loss: 0.4772 - val_accuracy: 0.7731
Epoch 4/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.4843 - accuracy: 0.7674 - val_loss: 0.4675 - val_accuracy: 0.7775
Epoch 5/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.4766 - accuracy: 0.7711 - val_loss: 0.4569 - val_accuracy: 0.7814
Epoch 6/40
3125/3125 [==============================] - 14s 5ms/step - loss: 0.4712 - accuracy: 0.7758 - val_loss: 0.4537 - val_accuracy: 0.7778
Epoch 7/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.4655 - accuracy: 0.7774 - val_loss: 0.4493 - val_ac

In [16]:
model.save(MODEL_OUTPUT_FILEPATH)

INFO:tensorflow:Assets written to: model_100K_glove_91_X\assets


INFO:tensorflow:Assets written to: model_100K_glove_91_X\assets


In [17]:
import random
#test_neg = random.choice(neg.values)
i = random.choice(test_dataset.index)
if test_dataset.loc[i]['target'] == '0':
    print('negative: ', test_dataset.loc[i]['text'])
else:
    print('positive: ', test_dataset.loc[i]['text'])
test_sequences = text.tokenizer.texts_to_sequences([test_dataset.loc[i]['text']])
test_padded2 = pad_sequences(
    test_sequences, maxlen=text.max_length, padding="post", truncating="post"
)
model.predict(test_padded2)

negative:  icing knee started swell stopped walking hurts also seriously feel like alone tonight


array([[0.01282087]], dtype=float32)

In [18]:
# import keras
# model = keras.models.load_model('C:/Users/Konst/Desktop/PythonNotebooks')

In [19]:
# model.summary()

In [20]:
#model.save('')